In [1]:
import pandas as pd
import numpy as np

from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json

import plotly.express as px

C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
df = pd.read_csv("ar41_with_speed_cleaned.csv",sep=";")
df['timestamps_UTC'] = pd.to_datetime(df['timestamps_UTC'])

In [8]:
dff = pd.read_csv('anomalies.csv', sep=';')
dff

,Unnamed: 0,mapped_veh_id,timestamps_UTC,lat,lon,RS_E_InAirTemp_PC1,RS_E_InAirTemp_PC2,RS_E_OilPress_PC1,RS_E_OilPress_PC2,RS_E_RPM_PC1,...,anomaly_RS_E_InAirTemp_PC1,anomaly_RS_T_OilTemp_PC1,anomaly_RS_E_OilPress_PC1,anomaly_RS_E_RPM_PC1,anomaly_RS_E_WatTemp_PC1,anomaly_RS_E_InAirTemp_PC2,anomaly_RS_T_OilTemp_PC2,anomaly_RS_E_OilPress_PC2,anomaly_RS_E_RPM_PC2,anomaly_RS_E_WatTemp_PC2
0,0,181.0,2023-01-23 08:28:54,51.040004,3.692637,13,11,269.0,241.0,798.0,...,No,No,No,No,No,No,No,No,No,No
1,1,181.0,2023-01-23 08:29:48,51.039857,3.693306,13,12,265.0,238.0,801.0,...,No,No,No,No,No,No,No,No,No,No
2,2,181.0,2023-01-23 08:29:54,51.040017,3.692645,13,12,265.0,248.0,798.0,...,No,No,No,No,No,No,No,No,No,No
3,3,181.0,2023-01-23 08:30:48,51.039853,3.693292,13,12,272.0,241.0,811.0,...,No,No,No,No,No,No,No,No,No,No
4,4,181.0,2023-01-23 08:30:55,51.039999,3.692651,13,12,269.0,241.0,791.0,...,No,No,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16867535,16867535,196.0,2023-09-04 00:27:36,50.420024,4.534313,27,27,386.0,386.0,792.0,...,No,Yes,No,No,Yes,No,Yes,No,No,Yes
16867536,16867536,196.0,2023-09-04 00:27:56,50.420183,4.534716,24,25,517.5,377.5,205.5,...,No,Yes,No,No,Yes,No,Yes,No,No,Yes
16867537,16867537,196.0,2023-09-08 09:40:09,50.420775,4.536776,42,42,13.0,13.0,0.0,...,No,Yes,No,No,Yes,No,Yes,No,No,Yes
16867538,16867538,196.0,2023-09-08 09:40:15,50.420509,4.536249,42,42,103.0,103.0,183.0,...,No,Yes,No,No,Yes,No,Yes,No,No,Yes


In [3]:
dataset_adapted = df[["timestamps_UTC",'mapped_veh_id','RS_E_InAirTemp_PC1',"RS_T_OilTemp_PC1","RS_E_OilPress_PC1","RS_E_RPM_PC1","RS_E_WatTemp_PC1","speed","outside_temp"]]
dataset_adapted.head()

,timestamps_UTC,mapped_veh_id,RS_E_InAirTemp_PC1,RS_T_OilTemp_PC1,RS_E_OilPress_PC1,RS_E_RPM_PC1,RS_E_WatTemp_PC1,speed,outside_temp
0,2023-08-01 03:44:12,181.0,27,76.0,255.0,794.0,83.0,3,15.4
1,2023-08-01 06:36:29,143.0,33,73.0,272.0,802.0,78.0,21,16.6
2,2023-08-24 06:53:54,183.0,31,85.0,234.0,799.0,82.0,2,18.7
3,2023-08-01 13:53:38,177.0,35,78.0,220.0,794.0,77.0,54,19.4
4,2023-08-24 07:02:30,143.0,41,82.0,227.0,806.0,85.0,2,18.7


In [ ]:
selected_column = ['RS_E_InAirTemp_PC1',"RS_T_OilTemp_PC1","RS_E_OilPress_PC1","RS_E_RPM_PC1","RS_E_WatTemp_PC1"]
anomalies_df = pd.DataFrame()

for veh_id in df['mapped_veh_id'].unique():
    print(veh_id)
    veh_anomalies = df[df['mapped_veh_id'] == veh_id]
    veh_anomalies = veh_anomalies.sort_values('timestamps_UTC')
    veh_anomalies = pd.DataFrame(veh_anomalies)
    for col in selected_column:
        dataset_adapted = df[["timestamps_UTC",'mapped_veh_id','RS_E_InAirTemp_PC1',"RS_T_OilTemp_PC1","RS_E_OilPress_PC1","RS_E_RPM_PC1","RS_E_WatTemp_PC1","speed","outside_temp"]]
        dataset_adapted = dataset_adapted.rename(columns={'timestamps_UTC': 'ds',col:'y'})
        model = Prophet()
        for col_reg in dataset_adapted.columns:
            if col_reg != "ds" and col_reg != "mapped_veh_id" and col_reg != "y":
                model.add_regressor(col_reg)

        # train
        train_veh = dataset_adapted[dataset_adapted['mapped_veh_id'] == veh_id].sort_values('ds')
        train_veh.drop(columns='mapped_veh_id')
        model.fit(train_veh)

        # predict
        future = model.make_future_dataframe(periods=0) # predict on the entire dataset
        for col_reg in dataset_adapted.columns:
            if col_reg != "ds" and col_reg != "mapped_veh_id" and col_reg != "y":
                future[col_reg] = train_veh[col_reg].reset_index(drop = True)
        forecast = model.predict(future)

        forecast['y'] = train_veh['y'].reset_index(drop = True)
        forecast['error'] = forecast['y'] - forecast['yhat']
        forecast['uncertainity'] = forecast['yhat_upper'] - forecast['yhat_lower']
        threshold  = 2
        forecast['anomaly'] = forecast.apply(lambda x: 'Yes' if ( np.abs(x['error']) > threshold * x['uncertainity'] ) else 'No', axis=1)
        veh_anomalies[f"anomaly_{col}"] = np.array(forecast['anomaly'])
    anomalies_df = pd.concat([anomalies_df, veh_anomalies])
        # fig = px.scatter(forecast.reset_index(), x='ds', y='y', color='anomaly', title='SNCB outliers')
        # fig.update_xaxes(rangeslider_visible=True,
        #                 rangeselector=dict(buttons=list([
        #                     dict(count=1, label='Ja+
    #                     nuary', step='month', stepmode='backward'),
        #                     dict(count=2, label='February', step='month', stepmode='backward'),
        #                     dict(count=3, label='March', step='month', stepmode='backward'),
        #                     dict(step='all')
        #                 ])))
        # fig.write_html(f"./plot/timeseries_anomalies_veh{int(veh_id)}.html")

In [2]:
df = pd.read_csv('../anomalies.csv', sep=';')

In [ ]:
selected_column = ['RS_E_InAirTemp_PC2',"RS_T_OilTemp_PC2","RS_E_OilPress_PC2","RS_E_RPM_PC2","RS_E_WatTemp_PC2"]
anomalies_df = pd.DataFrame()

for veh_id in df['mapped_veh_id'].unique():
    veh_anomalies = df[df['mapped_veh_id'] == veh_id]
    veh_anomalies = veh_anomalies.sort_values('timestamps_UTC')
    veh_anomalies = pd.DataFrame(veh_anomalies)
    for col in selected_column:
        dataset_adapted = df[["timestamps_UTC",'mapped_veh_id','RS_E_InAirTemp_PC2',"RS_T_OilTemp_PC2",
                              "RS_E_OilPress_PC2","RS_E_RPM_PC2","RS_E_WatTemp_PC2","speed","outside_temp"]]
        dataset_adapted = dataset_adapted.rename(columns={'timestamps_UTC': 'ds',col:'y'})
        model = Prophet()
        for col_reg in dataset_adapted.columns:
            if col_reg != "ds" and col_reg != "mapped_veh_id" and col_reg != "y":
                model.add_regressor(col_reg)

        # train
        train_veh = dataset_adapted[dataset_adapted['mapped_veh_id'] == veh_id].sort_values('ds')
        train_veh.drop(columns='mapped_veh_id')
        model.fit(train_veh)

        # predict
        future = model.make_future_dataframe(periods=0) # predict on the entire dataset
        for col_reg in dataset_adapted.columns:
            if col_reg != "ds" and col_reg != "mapped_veh_id" and col_reg != "y":
                future[col_reg] = train_veh[col_reg].reset_index(drop = True)

        forecast = model.predict(future)
        forecast['y'] = train_veh['y'].reset_index(drop = True)
        forecast['error'] = forecast['y'] - forecast['yhat']
        forecast['uncertainity'] = forecast['yhat_upper'] - forecast['yhat_lower']
        threshold  = 2
        forecast['anomaly'] = forecast.apply(lambda x: 'Yes' if ( np.abs(x['error']) > threshold * x['uncertainity'] ) else 'No', axis=1)
        veh_anomalies[f"anomaly_{col}"] = np.array(forecast['anomaly'])


    anomalies_df = pd.concat([anomalies_df, veh_anomalies])


In [7]:
anomalies_df

,mapped_veh_id,timestamps_UTC,lat,lon,RS_E_InAirTemp_PC1,RS_E_InAirTemp_PC2,RS_E_OilPress_PC1,RS_E_OilPress_PC2,RS_E_RPM_PC1,RS_E_RPM_PC2,...,anomaly_RS_E_InAirTemp_PC1,anomaly_RS_T_OilTemp_PC1,anomaly_RS_E_OilPress_PC1,anomaly_RS_E_RPM_PC1,anomaly_RS_E_WatTemp_PC1,anomaly_RS_E_InAirTemp_PC2,anomaly_RS_T_OilTemp_PC2,anomaly_RS_E_OilPress_PC2,anomaly_RS_E_RPM_PC2,anomaly_RS_E_WatTemp_PC2
0,181.0,2023-01-23 08:28:54,51.040004,3.692637,13,11,269.0,241.0,798.0,799.0,...,No,No,No,No,No,No,No,No,No,No
1,181.0,2023-01-23 08:29:48,51.039857,3.693306,13,12,265.0,238.0,801.0,799.0,...,No,No,No,No,No,No,No,No,No,No
2,181.0,2023-01-23 08:29:54,51.040017,3.692645,13,12,265.0,248.0,798.0,799.0,...,No,No,No,No,No,No,No,No,No,No
3,181.0,2023-01-23 08:30:48,51.039853,3.693292,13,12,272.0,241.0,811.0,799.0,...,No,No,No,No,No,No,No,No,No,No
4,181.0,2023-01-23 08:30:55,51.039999,3.692651,13,12,269.0,241.0,791.0,798.0,...,No,No,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16867535,196.0,2023-09-04 00:27:36,50.420024,4.534313,27,27,386.0,386.0,792.0,792.0,...,No,Yes,No,No,Yes,No,Yes,No,No,Yes
16867536,196.0,2023-09-04 00:27:56,50.420183,4.534716,24,25,517.5,377.5,205.5,798.0,...,No,Yes,No,No,Yes,No,Yes,No,No,Yes
16867537,196.0,2023-09-08 09:40:09,50.420775,4.536776,42,42,13.0,13.0,0.0,0.0,...,No,Yes,No,No,Yes,No,Yes,No,No,Yes
16867538,196.0,2023-09-08 09:40:15,50.420509,4.536249,42,42,103.0,103.0,183.0,183.0,...,No,Yes,No,No,Yes,No,Yes,No,No,Yes


In [5]:
anomalies_df = anomalies_df.drop(columns='Unnamed: 0')

In [6]:
anomalies_df.to_csv('anomalies.csv',sep=';')

In [2]:
anomalies_df = pd.read_csv('anomalies.csv', sep=';')

In [5]:
len(anomalies_df)

16867540

In [11]:
anomalies_df = anomalies_df.drop(columns=['Unnamed: 0','Unnamed: 0.1'])

In [13]:
anomalies_df[anomalies_df['mapped_veh_id']==182]

,mapped_veh_id,timestamps_UTC,lat,lon,RS_E_InAirTemp_PC1,RS_E_InAirTemp_PC2,RS_E_OilPress_PC1,RS_E_OilPress_PC2,RS_E_RPM_PC1,RS_E_RPM_PC2,...,station_name,province_name,outside_temp,weather,speed,anomaly_RS_E_InAirTemp_PC1,anomaly_RS_T_OilTemp_PC1,anomaly_RS_E_OilPress_PC1,anomaly_RS_E_RPM_PC1,anomaly_RS_E_WatTemp_PC1
2662100,182.0,2023-01-23 08:24:04,51.035656,3.710717,16,9,207.0,224.0,806.0,799.0,...,Ghent Sint Pieters,Oost-Vlaanderen,1.7,Broken clouds,0,No,No,No,No,No
2662101,182.0,2023-01-23 08:24:56,51.035410,3.711797,16,8,210.0,224.0,797.0,799.0,...,Ghent Sint Pieters,Oost-Vlaanderen,1.7,Broken clouds,5,No,No,No,No,No
2662102,182.0,2023-01-23 08:25:57,51.035407,3.711814,17,8,210.0,224.0,798.0,798.0,...,Ghent Sint Pieters,Oost-Vlaanderen,1.7,Broken clouds,0,No,No,No,No,No
2662103,182.0,2023-01-23 08:30:00,51.035400,3.711835,17,7,217.0,227.0,802.0,799.0,...,Ghent Sint Pieters,Oost-Vlaanderen,1.7,Broken clouds,0,No,No,No,No,No
2662104,182.0,2023-01-23 08:31:00,51.035399,3.711808,16,7,220.0,231.0,801.0,802.0,...,Ghent Sint Pieters,Oost-Vlaanderen,1.9,Broken clouds,0,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2792964,182.0,2023-09-13 16:24:15,50.768640,3.870845,31,36,179.0,179.0,803.0,801.0,...,Geraardsbergen,Oost-Vlaanderen,19.7,Broken clouds,21,No,No,No,No,No
2792965,182.0,2023-09-13 16:25:16,50.771994,3.872430,33,35,179.0,175.0,799.0,800.0,...,Geraardsbergen,Oost-Vlaanderen,19.7,Broken clouds,22,No,No,No,No,No
2792966,182.0,2023-09-13 16:26:16,50.774384,3.872867,33,35,179.0,179.0,799.0,799.0,...,NaN,Oost-Vlaanderen,19.7,Broken clouds,16,No,No,No,No,No
2792967,182.0,2023-09-13 16:26:37,50.774389,3.872878,35,35,17.0,51.0,142.0,142.0,...,NaN,Oost-Vlaanderen,19.7,Broken clouds,0,No,No,No,No,No


In [26]:
(len(forecast[forecast['anomaly'] == 'Yes'])/len(forecast))*100

0.35511925899955893

In [24]:
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

In [25]:
cols = [['RS_E_InAirTemp_PC1', 'outside_temp'],['RS_E_OilPress_PC1','RS_E_RPM_PC1'], ['RS_E_WatTemp_PC1','RS_T_OilTemp_PC1']]

In [26]:
anomalies_df = pd.DataFrame()
for veh_id in df['mapped_veh_id'].unique():
    print(veh_id)
    veh_df = pd.DataFrame(df[df['mapped_veh_id'] == veh_id])
    for selected_column in cols:
        X = veh_df[selected_column]
        isolation_forest= IsolationForest(n_estimators=500, contamination=0.001, random_state=42)
        pred = np.array(isolation_forest.fit_predict(X))
        veh_df['anomaly_'+selected_column[0]] = pred
        veh_df['anomaly_'+selected_column[1]] = pred
    anomalies_df = pd.concat([anomalies_df, veh_df])


181.0
143.0
183.0
177.0
121.0
136.0
172.0
180.0
151.0
128.0
182.0
194.0
134.0
140.0
161.0
142.0
166.0
176.0
178.0
145.0
125.0
152.0
154.0
109.0
146.0
170.0
106.0
188.0
157.0
103.0
171.0
138.0
137.0
112.0
179.0
190.0
122.0
102.0
141.0
189.0
173.0
116.0
114.0
159.0
153.0
149.0
111.0
150.0
184.0
126.0
135.0
192.0
160.0
175.0
119.0
164.0
186.0
174.0
187.0
167.0
108.0
133.0
168.0
110.0
191.0
124.0
130.0
185.0
105.0
120.0
158.0
197.0
162.0
115.0
104.0
139.0
169.0
155.0
165.0
147.0
156.0
117.0
113.0
144.0
129.0
107.0
123.0
163.0
127.0
148.0
131.0
196.0


In [29]:
df = anomalies_df

In [30]:
cols = [['RS_E_InAirTemp_PC2', 'outside_temp'],['RS_E_OilPress_PC2','RS_E_RPM_PC2'], ['RS_E_WatTemp_PC2','RS_T_OilTemp_PC2']]

In [31]:
anomalies_df = pd.DataFrame()
for veh_id in df['mapped_veh_id'].unique():
    print(veh_id)
    veh_df = pd.DataFrame(df[df['mapped_veh_id'] == veh_id])
    for selected_column in cols:
        X = veh_df[selected_column]
        isolation_forest= IsolationForest(n_estimators=500, contamination=0.01, random_state=42)
        pred = np.array(isolation_forest.fit_predict(X))
        veh_df['anomaly_'+selected_column[0]] = pred
        veh_df['anomaly_'+selected_column[1]] = pred
    anomalies_df = pd.concat([anomalies_df, veh_df])


181.0
143.0
183.0
177.0
121.0
136.0
172.0
180.0
151.0
128.0
182.0
194.0
134.0
140.0
161.0
142.0
166.0
176.0
178.0
145.0
125.0
152.0
154.0
109.0
146.0
170.0
106.0
188.0
157.0
103.0
171.0
138.0
137.0
112.0
179.0
190.0
122.0
102.0
141.0
189.0
173.0
116.0
114.0
159.0
153.0
149.0
111.0
150.0
184.0
126.0
135.0
192.0
160.0
175.0
119.0
164.0
186.0
174.0
187.0
167.0
108.0
133.0
168.0
110.0
191.0
124.0
130.0
185.0
105.0
120.0
158.0
197.0
162.0
115.0
104.0
139.0
169.0
155.0
165.0
147.0
156.0
117.0
113.0
144.0
129.0
107.0
123.0
163.0
127.0
148.0
131.0
196.0


In [33]:
anomalies_df = anomalies_df.drop(columns=['anomaly_outside_temp'])

In [36]:
anomalies_df_bis = anomalies_df.copy()
for col in anomalies_df.columns:
    if 'anomaly_' in col:
        print(col)
        anomalies_df_bis[col] = anomalies_df.apply(lambda x: 'Yes' if (x[col] == -1 ) else 'No', axis=1)

anomaly_RS_E_InAirTemp_PC1
anomaly_RS_E_OilPress_PC1
anomaly_RS_E_RPM_PC1
anomaly_RS_E_WatTemp_PC1
anomaly_RS_T_OilTemp_PC1
anomaly_RS_E_InAirTemp_PC2
anomaly_RS_E_OilPress_PC2
anomaly_RS_E_RPM_PC2
anomaly_RS_E_WatTemp_PC2
anomaly_RS_T_OilTemp_PC2


In [2]:
anomalies_df_bis.to_csv('anomalies.csv', sep=';')

NameError: name 'anomalies_df_bis' is not defined

In [2]:
df = pd.read_csv('anomalies.csv', sep=';')

In [3]:
demo = df[(df['mapped_veh_id'] == 180) | (df['mapped_veh_id'] == 181) | (df['mapped_veh_id'] == 163) | (df['mapped_veh_id'] == 142)]
demo

,Unnamed: 0,mapped_veh_id,timestamps_UTC,lat,lon,RS_E_InAirTemp_PC1,RS_E_InAirTemp_PC2,RS_E_OilPress_PC1,RS_E_OilPress_PC2,RS_E_RPM_PC1,...,anomaly_RS_E_InAirTemp_PC1,anomaly_RS_T_OilTemp_PC1,anomaly_RS_E_OilPress_PC1,anomaly_RS_E_RPM_PC1,anomaly_RS_E_WatTemp_PC1,anomaly_RS_E_InAirTemp_PC2,anomaly_RS_T_OilTemp_PC2,anomaly_RS_E_OilPress_PC2,anomaly_RS_E_RPM_PC2,anomaly_RS_E_WatTemp_PC2
0,0,181.0,2023-01-23 08:28:54,51.040004,3.692637,13,11,269.0,241.0,798.0,...,No,No,No,No,No,No,No,No,No,No
1,1,181.0,2023-01-23 08:29:48,51.039857,3.693306,13,12,265.0,238.0,801.0,...,No,No,No,No,No,No,No,No,No,No
2,2,181.0,2023-01-23 08:29:54,51.040017,3.692645,13,12,265.0,248.0,798.0,...,No,No,No,No,No,No,No,No,No,No
3,3,181.0,2023-01-23 08:30:48,51.039853,3.693292,13,12,272.0,241.0,811.0,...,No,No,No,No,No,No,No,No,No,No
4,4,181.0,2023-01-23 08:30:55,51.039999,3.692651,13,12,269.0,241.0,791.0,...,No,No,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16645378,16645378,163.0,2023-06-13 12:31:36,50.467596,4.840690,42,40,220.0,341.0,800.0,...,No,No,No,No,No,No,No,No,No,No
16645379,16645379,163.0,2023-06-13 12:32:36,50.467975,4.840861,41,40,220.0,272.0,805.0,...,No,No,No,No,No,No,No,No,No,No
16645380,16645380,163.0,2023-06-13 12:33:36,50.468019,4.840877,41,40,62.0,120.0,106.0,...,No,No,No,No,No,No,No,No,No,No
16645381,16645381,163.0,2023-06-13 12:34:36,50.468060,4.840822,50,48,10.0,10.0,0.0,...,No,No,No,No,No,No,No,No,No,No


In [4]:
demo.to_csv('demo.csv', sep=';')

In [5]:
demo[(demo['RS_E_InAirTemp_PC1'] > 65) & (demo['anomaly_RS_E_InAirTemp_PC1'] == 'Yes')]

,Unnamed: 0,mapped_veh_id,timestamps_UTC,lat,lon,RS_E_InAirTemp_PC1,RS_E_InAirTemp_PC2,RS_E_OilPress_PC1,RS_E_OilPress_PC2,RS_E_RPM_PC1,...,anomaly_RS_E_InAirTemp_PC1,anomaly_RS_T_OilTemp_PC1,anomaly_RS_E_OilPress_PC1,anomaly_RS_E_RPM_PC1,anomaly_RS_E_WatTemp_PC1,anomaly_RS_E_InAirTemp_PC2,anomaly_RS_T_OilTemp_PC2,anomaly_RS_E_OilPress_PC2,anomaly_RS_E_RPM_PC2,anomaly_RS_E_WatTemp_PC2
21290,21290,181.0,2023-02-02 22:54:21,51.015642,3.775396,66,53,20.0,20.0,0.0,...,Yes,No,No,No,No,No,No,No,No,No
21291,21291,181.0,2023-02-02 22:54:24,51.015391,3.775883,66,53,20.0,20.0,0.0,...,Yes,No,No,No,No,No,No,No,No,No
21292,21292,181.0,2023-02-02 22:55:20,51.015637,3.775392,66,55,20.0,20.0,0.0,...,Yes,No,No,No,No,No,No,No,No,No
21293,21293,181.0,2023-02-02 22:55:24,51.015391,3.775935,66,55,20.0,20.0,0.0,...,Yes,No,No,No,No,No,No,No,No,No
21294,21294,181.0,2023-02-02 22:56:17,51.015446,3.775993,66,56,20.0,20.0,0.0,...,Yes,No,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16630110,16630110,163.0,2023-05-26 21:49:13,51.189609,5.109427,68,62,10.0,10.0,0.0,...,Yes,No,No,No,No,No,No,No,No,No
16630111,16630111,163.0,2023-05-26 21:50:12,51.189614,5.109413,69,62,10.0,10.0,0.0,...,Yes,No,No,No,No,No,No,No,No,No
16630112,16630112,163.0,2023-05-26 21:51:12,51.189596,5.109437,69,63,10.0,10.0,0.0,...,Yes,No,No,No,No,No,No,No,No,No
16632725,16632725,163.0,2023-05-29 22:47:44,51.189595,5.109420,66,62,10.0,10.0,0.0,...,Yes,No,No,No,No,No,No,No,No,No
